In [ ]:
import torch
print(torch.cuda.is_available())
print(torch.cuda.get_device_name(0))


True
Tesla T4


In [ ]:
!nvidia-smi


Wed Dec 24 12:51:56 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   37C    P8              9W /   70W |       2MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [ ]:
import torch

x = torch.randn(5000, 5000, device="cuda")
y = torch.randn(5000, 5000, device="cuda")
z = x @ y
print(z.device)


cuda:0


In [ ]:
!nvidia-smi


Wed Dec 24 12:53:19 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   39C    P0             25W /   70W |     518MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [ ]:
!git clone https://github.com/NVIDIA/cuda-samples.git
%cd cuda-samples


Cloning into 'cuda-samples'...
remote: Enumerating objects: 30467, done.
remote: Total 30467 (delta 0), reused 0 (delta 0), pack-reused 30467 (from 1)
Receiving objects: 100% (30467/30467), 137.01 MiB | 12.70 MiB/s, done.
Resolving deltas: 100% (27067/27067), done.
Updating files: 100% (2052/2052), done.
/content/cuda-samples


In [ ]:
!apt-get install -y cmake


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
cmake is already the newest version (3.22.1-1ubuntu1.22.04.2).
0 upgraded, 0 newly installed, 0 to remove and 41 not upgraded.


In [ ]:
!mkdir build

In [ ]:
%cd build


/content/cuda-samples/build


In [ ]:
cd ..

/content/cuda-samples


In [ ]:
!find Samples -type d -name "*vectorAdd*"

Samples/0_Introduction/vectorAdd_nvrtc
Samples/0_Introduction/vectorAdd
Samples/0_Introduction/vectorAddDrv
Samples/0_Introduction/vectorAddMMAP


In [ ]:
!nvcc vectorAdd.cu -I../../../Common -o vectorAdd


In [ ]:
!ls


CMakeLists.txt	README.md  vectorAdd  vectorAdd.cu


In [ ]:
!./vectorAdd


[Vector addition of 50000 elements]
Copy input data from the host memory to the CUDA device
CUDA kernel launch with 196 blocks of 256 threads
Failed to launch vectorAdd kernel (error code the provided PTX was compiled with an unsupported toolchain.)!


In [ ]:
!nvcc vectorAdd.cu -I../../../Common --gpu-architecture=native -o vectorAdd


In [ ]:
!./vectorAdd

[Vector addition of 50000 elements]
Copy input data from the host memory to the CUDA device
CUDA kernel launch with 196 blocks of 256 threads
Copy output data from the CUDA device to the host memory
Test PASSED
Done


In [ ]:
%cd /content/cuda-samples/Samples/0_Introduction/matrixMul
!ls


/content/cuda-samples/Samples/0_Introduction/matrixMul
CMakeLists.txt	matrixMul.cu  README.md


In [ ]:
!nvcc matrixMul.cu \
-I../../../Common \
--gpu-architecture=native \
-o matrixMul


In [ ]:
!./matrixMul

[Matrix Multiply Using CUDA] - Starting...
GPU Device 0: "Turing" with compute capability 7.5

MatrixA(320,320), MatrixB(640,320)
Computing result using CUDA Kernel...
done
Performance= 410.85 GFlop/s, Time= 0.319 msec, Size= 131072000 Ops, WorkgroupSize= 1024 threads/block
Checking computed result for correctness: Result = PASS

NOTE: The CUDA Samples are not meant for performance measurements. Results may vary when GPU Boost is enabled.


In [ ]:
%cd /content
!pwd
!mkdir -p week2
!ls


/content
/content
cuda-samples  sample_data  week2


In [ ]:
%%writefile vector_add.cu
#include <cuda_runtime.h>
#include <iostream>
#include <vector>
#include <cmath>
__global__ void vectorAdd(const float* A, const float* B, float* C, int N) {
    int i = blockIdx.x * blockDim.x + threadIdx.x;
    if (i < N) {
        C[i] = A[i] + B[i];
    }
}
int main() {
    int N = 1024;
    std::vector<float> h_A(N), h_B(N), h_C(N);
    for (int i = 0; i < N; i++) {
        h_A[i] = i * 1.0f;
        h_B[i] = 2.0f * i;
    }
    float *d_A, *d_B, *d_C;
    cudaMalloc(&d_A, N * sizeof(float));
    cudaMalloc(&d_B, N * sizeof(float));
    cudaMalloc(&d_C, N * sizeof(float));
    cudaMemcpy(d_A, h_A.data(), N * sizeof(float), cudaMemcpyHostToDevice);
    cudaMemcpy(d_B, h_B.data(), N * sizeof(float), cudaMemcpyHostToDevice);
    int threads = 256;
    int blocks = (N + threads - 1) / threads;
    vectorAdd<<<blocks, threads>>>(d_A, d_B, d_C, N);
    cudaMemcpy(h_C.data(), d_C, N * sizeof(float), cudaMemcpyDeviceToHost);
    for (int i = 0; i < N; i++) {
        float expected = h_A[i] + h_B[i];
        if (std::fabs(h_C[i] - expected) > 1e-5) {
            std::cout << "FAIL at " << i << std::endl;
            return 1;
        }
    }
    std::cout << "Vector Add: PASS" << std::endl;
    cudaFree(d_A);
    cudaFree(d_B);
    cudaFree(d_C);
    return 0;
}



Overwriting vector_add.cu


In [ ]:
!nvcc vector_add.cu --gpu-architecture=native -o vector_add
!./vector_add


Vector Add: PASS


In [ ]:
!pwd


/content


In [ ]:
%%writefile multiply_scale.cu
#include <cuda_runtime.h>
#include <iostream>
#include <vector>
#include <cmath>
__global__ void multiplyScale(const float* A, const float* B, float* C, float alpha, int N) {
    int i = blockIdx.x * blockDim.x + threadIdx.x;
    if (i < N) {
        C[i] = alpha * A[i] * B[i];
    }
}
int main() {
    int N = 1024;
    float alpha = 2.0f;

    std::vector<float> h_A(N), h_B(N), h_C(N);

    for (int i = 0; i < N; i++) {
        h_A[i] = i * 1.0f;
        h_B[i] = i * 0.5f;
    }

    float *d_A, *d_B, *d_C;
    cudaMalloc(&d_A, N * sizeof(float));
    cudaMalloc(&d_B, N * sizeof(float));
    cudaMalloc(&d_C, N * sizeof(float));

    cudaMemcpy(d_A, h_A.data(), N * sizeof(float), cudaMemcpyHostToDevice);
    cudaMemcpy(d_B, h_B.data(), N * sizeof(float), cudaMemcpyHostToDevice);

    int threads = 256;
    int blocks = (N + threads - 1) / threads;
    multiplyScale<<<blocks, threads>>>(d_A, d_B, d_C, alpha, N);

    cudaMemcpy(h_C.data(), d_C, N * sizeof(float), cudaMemcpyDeviceToHost);

    for (int i = 0; i < N; i++) {
        float expected = alpha * h_A[i] * h_B[i];
        if (std::fabs(h_C[i] - expected) > 1e-5) {
            std::cout << "FAIL at index " << i << std::endl;
            return 1;
        }
    }
    std::cout << "Multiply-and-Scale: PASS" << std::endl;
    cudaFree(d_A);
    cudaFree(d_B);
    cudaFree(d_C);
    return 0;
}


Writing multiply_scale.cu


In [ ]:
!nvcc multiply_scale.cu --gpu-architecture=native -o multiply_scale
!./multiply_scale


Multiply-and-Scale: PASS


In [ ]:
!pwd


/content


In [ ]:
%%writefile relu.cu
#include <cuda_runtime.h>
#include <iostream>
#include <vector>
#include <cmath>
__global__ void reluKernel(const float* A, float* C, int N) {
    int i = blockIdx.x * blockDim.x + threadIdx.x;
    if (i < N) {
        C[i] = (A[i] > 0.0f) ? A[i] : 0.0f;
    }
}
int main() {
    int N = 1024;
    std::vector<float> h_A(N), h_C(N);
    for (int i = 0; i < N; i++) {
        h_A[i] = i - 512;   // some negatives, some positives
    }
    float *d_A, *d_C;
    cudaMalloc(&d_A, N * sizeof(float));
    cudaMalloc(&d_C, N * sizeof(float));
    cudaMemcpy(d_A, h_A.data(), N * sizeof(float), cudaMemcpyHostToDevice);
    int threads = 256;
    int blocks = (N + threads - 1) / threads;
    reluKernel<<<blocks, threads>>>(d_A, d_C, N);
    cudaMemcpy(h_C.data(), d_C, N * sizeof(float), cudaMemcpyDeviceToHost);
    for (int i = 0; i < N; i++) {
        float expected = std::max(h_A[i], 0.0f);
        if (std::fabs(h_C[i] - expected) > 1e-5) {
            std::cout << "FAIL at index " << i << std::endl;
            return 1;
        }
    }
    std::cout << "ReLU: PASS" << std::endl;
    cudaFree(d_A);
    cudaFree(d_C);
    return 0;
}


Writing relu.cu


In [ ]:
!nvcc relu.cu --gpu-architecture=native -o relu
!./relu


ReLU: PASS


**Ignore**

In [ ]:
!ls


cuda-samples	multiply_scale.cu  relu.cu	vector_add     week2
multiply_scale	relu		   sample_data	vector_add.cu


In [ ]:
cd week2


/content/week2


In [ ]:
!ls


In [ ]:
!rm -r week2


rm: cannot remove 'week2': No such file or directory


In [ ]:
cd ..

/content


In [ ]:
!rm -r week2


In [ ]:
!rm vector_add multiply_scale relu


In [ ]:
!ls

cuda-samples  multiply_scale.cu  relu.cu  sample_data  vector_add.cu


In [ ]:
%cd /content
!ls


/content
cuda-samples  multiply_scale.cu  relu.cu  sample_data  vector_add.cu


**Task2**


In [ ]:
%%writefile vector_add.cu
#include <cuda_runtime.h>
#include <iostream>
#include <vector>
#include <cmath>
__global__ void vectorAdd(const float* A, const float* B, float* C, int N) {
    int i = blockIdx.x * blockDim.x + threadIdx.x;
    if (i < N) {
        C[i] = A[i] + B[i];
    }
}
int main() {
    int N = 10000000;
    std::vector<float> h_A(N), h_B(N), h_C(N);
    for (int i = 0; i < N; i++) {
        h_A[i] = i * 1.0f;
        h_B[i] = 2.0f * i;
    }
    float *d_A, *d_B, *d_C;
    cudaMalloc(&d_A, N * sizeof(float));
    cudaMalloc(&d_B, N * sizeof(float));
    cudaMalloc(&d_C, N * sizeof(float));
    cudaMemcpy(d_A, h_A.data(), N * sizeof(float), cudaMemcpyHostToDevice);
    cudaMemcpy(d_B, h_B.data(), N * sizeof(float), cudaMemcpyHostToDevice);
    int threads = 32;
    int blocks = (N + threads - 1) / threads;
    vectorAdd<<<blocks, threads>>>(d_A, d_B, d_C, N);
    cudaMemcpy(h_C.data(), d_C, N * sizeof(float), cudaMemcpyDeviceToHost);
    for (int i = 0; i < N; i++) {
        float expected = h_A[i] + h_B[i];
        if (std::fabs(h_C[i] - expected) > 1e-5) {
            std::cout << "FAIL at " << i << std::endl;
            return 1;
        }
    }
    std::cout << "Vector Add: PASS" << std::endl;
    std::cout << "N = " << N << std::endl;
    std::cout << "blockDim.x = " << threads << std::endl;
    std::cout << "gridDim.x = " << blocks << std::endl;
    std::cout << "Total threads launched = " << blocks * threads << std::endl;
    cudaFree(d_A);
    cudaFree(d_B);
    cudaFree(d_C);
    return 0;
}




Overwriting vector_add.cu


In [ ]:
!nvcc vector_add.cu --gpu-architecture=native -o vector_add
!./vector_add


Vector Add: PASS
N = 10000000
blockDim.x = 32
gridDim.x = 312500
Total threads launched = 10000000


**Task3**

**Observation Table1**

In [ ]:
%%writefile vector_add.cu
#include <cuda_runtime.h>
#include <iostream>
#include <vector>
#include <cmath>
__global__ void vectorAdd(const float* A, const float* B, float* C, int N) {
    int i = blockIdx.x * blockDim.x + threadIdx.x;
    if (i < N) {
        C[i] = A[i] + B[i];
    }
}
int main() {
    int N = 1000;
    std::vector<float> h_A(N), h_B(N), h_C(N);
    for (int i = 0; i < N; i++) {
        h_A[i] = i * 1.0f;
        h_B[i] = 2.0f * i;
    }
    float *d_A, *d_B, *d_C;
    cudaMalloc(&d_A, N * sizeof(float));
    cudaMalloc(&d_B, N * sizeof(float));
    cudaMalloc(&d_C, N * sizeof(float));
    cudaMemcpy(d_A, h_A.data(), N * sizeof(float), cudaMemcpyHostToDevice);
    cudaMemcpy(d_B, h_B.data(), N * sizeof(float), cudaMemcpyHostToDevice);
    int threads = 32;
    int blocks = (N + threads - 1) / threads;
    cudaEvent_t start, stop;
    cudaEventCreate(&start);
    cudaEventCreate(&stop);
    cudaEventRecord(start);
    vectorAdd<<<blocks, threads>>>(d_A, d_B, d_C, N);
    cudaEventRecord(stop);
    cudaEventSynchronize(stop);
    float milliseconds = 0;
    cudaEventElapsedTime(&milliseconds, start, stop);
    std::cout << "Kernel execution time (ms): "
              << milliseconds << std::endl;
    cudaEventDestroy(start);
    cudaEventDestroy(stop);
    cudaMemcpy(h_C.data(), d_C, N * sizeof(float), cudaMemcpyDeviceToHost);

    for (int i = 0; i < N; i++) {
        float expected = h_A[i] + h_B[i];
        if (std::fabs(h_C[i] - expected) > 1e-5) {
            std::cout << "FAIL at " << i << std::endl;
            return 1;
        }
    }

    std::cout << "Vector Add: PASS" << std::endl;
    std::cout << "N = " << N << std::endl;
    std::cout << "blockDim.x = " << threads << std::endl;
    std::cout << "gridDim.x = " << blocks << std::endl;
    std::cout << "Total threads launched = " << blocks * threads << std::endl;


    cudaFree(d_A);
    cudaFree(d_B);
    cudaFree(d_C);

    return 0;
}




Overwriting vector_add.cu


In [ ]:
!nvcc vector_add.cu --gpu-architecture=native -o vector_add
!./vector_add


Kernel execution time (ms): 0.102048
Vector Add: PASS
N = 1000
blockDim.x = 32
gridDim.x = 32
Total threads launched = 1024


**Observation Table2**

In [ ]:
%%writefile vector_add.cu
#include <cuda_runtime.h>
#include <iostream>
#include <vector>
#include <cmath>
__global__ void vectorAdd(const float* A, const float* B, float* C, int N) {
    int i = blockIdx.x * blockDim.x + threadIdx.x;
    if (i < N) {
        C[i] = A[i] + B[i];
    }
}
int main() {
    int Ns[] = {1000, 100000, 10000000};
    int blockSizes[] = {32, 128, 256, 512};
    for (int ni = 0; ni < 3; ni++) {
        int N = Ns[ni];
        std::vector<float> h_A(N), h_B(N), h_C(N);
        for (int i = 0; i < N; i++) {
            h_A[i] = i * 1.0f;
            h_B[i] = 2.0f * i;
        }
        float *d_A, *d_B, *d_C;
        cudaMalloc(&d_A, N * sizeof(float));
        cudaMalloc(&d_B, N * sizeof(float));
        cudaMalloc(&d_C, N * sizeof(float));
        cudaMemcpy(d_A, h_A.data(), N * sizeof(float), cudaMemcpyHostToDevice);
        cudaMemcpy(d_B, h_B.data(), N * sizeof(float), cudaMemcpyHostToDevice);
        for (int bi = 0; bi < 4; bi++) {
            int threads = blockSizes[bi];
            int blocks = (N + threads - 1) / threads;
            cudaEvent_t start, stop;
            cudaEventCreate(&start);
            cudaEventCreate(&stop);
            cudaEventRecord(start);
            vectorAdd<<<blocks, threads>>>(d_A, d_B, d_C, N);
            cudaEventRecord(stop);
            cudaEventSynchronize(stop);
            float milliseconds = 0;
            cudaEventElapsedTime(&milliseconds, start, stop);
            cudaMemcpy(h_C.data(), d_C, N * sizeof(float), cudaMemcpyDeviceToHost);
            bool pass = true;
            for (int i = 0; i < N; i++) {
                float expected = h_A[i] + h_B[i];
                if (std::fabs(h_C[i] - expected) > 1e-5) {
                    pass = false;
                    break;
                }
            }

            std::cout << "----------------------------------\n";
            std::cout << "N = " << N << "\n";
            std::cout << "blockDim.x = " << threads << "\n";
            std::cout << "gridDim.x = " << blocks << "\n";
            std::cout << "Total threads launched = "
                      << blocks * threads << "\n";
            std::cout << "Kernel execution time (ms) = "
                      << milliseconds << "\n";
            std::cout << "Result = "
                      << (pass ? "PASS" : "FAIL") << "\n";
            cudaEventDestroy(start);
            cudaEventDestroy(stop);
        }

        cudaFree(d_A);
        cudaFree(d_B);
        cudaFree(d_C);
    }

    return 0;
}


Writing vector_add.cu


In [ ]:
!nvcc vector_add.cu --gpu-architecture=native -o vector_add
!./vector_add


----------------------------------
N = 1000
blockDim.x = 32
gridDim.x = 32
Total threads launched = 1024
Kernel execution time (ms) = 0.099936
Result = PASS
----------------------------------
N = 1000
blockDim.x = 128
gridDim.x = 8
Total threads launched = 1024
Kernel execution time (ms) = 0.014304
Result = PASS
----------------------------------
N = 1000
blockDim.x = 256
gridDim.x = 4
Total threads launched = 1024
Kernel execution time (ms) = 0.010816
Result = PASS
----------------------------------
N = 1000
blockDim.x = 512
gridDim.x = 2
Total threads launched = 1024
Kernel execution time (ms) = 0.009888
Result = PASS
----------------------------------
N = 100000
blockDim.x = 32
gridDim.x = 3125
Total threads launched = 100000
Kernel execution time (ms) = 0.016192
Result = PASS
----------------------------------
N = 100000
blockDim.x = 128
gridDim.x = 782
Total threads launched = 100096
Kernel execution time (ms) = 0.01536
Result = PASS
----------------------------------
N = 100000
b